필요한 패키지 import

In [1]:
import pyupbit as pu
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

여기서 데이터 업데이트 하여서 dataset finder에 krw_btc.csv에 데이터 추가하기

In [4]:
df = pu.get_ohlcv("KRW-BTC", interval='minute60',count=100)
a = "krw-krw"
output = 'dataset/'+ str(a) + '.csv'
df.to_csv(output)

업데이트한 krw_btc.csv에 null 데이터가 존재하는지 확인하는 작업(null 데이터 있으면 찾아서 이전값으로 추가한다.)

In [17]:
df = pd.read_csv("dataset/krw_btc.csv")
df_dict = {}
for i in range(len(df)):
    if df["Unnamed: 0"][i][:10] in df_dict:
        df_dict[df["Unnamed: 0"][i][:10]]+=1
    else:
        df_dict[df["Unnamed: 0"][i][:10]]=1

sorted_dict = sorted(df_dict.items(), key= lambda item:item[1], reverse=False)
print(sorted_dict)

[('2017-09-26', 24), ('2017-09-27', 24), ('2017-09-28', 24), ('2017-09-29', 24), ('2017-09-30', 24), ('2017-10-01', 24), ('2017-10-02', 24), ('2017-10-03', 24), ('2017-10-04', 24), ('2017-10-05', 24), ('2017-10-06', 24), ('2017-10-07', 24), ('2017-10-08', 24), ('2017-10-09', 24), ('2017-10-10', 24), ('2017-10-11', 24), ('2017-10-12', 24), ('2017-10-13', 24), ('2017-10-14', 24), ('2017-10-15', 24), ('2017-10-16', 24), ('2017-10-17', 24), ('2017-10-18', 24), ('2017-10-19', 24), ('2017-10-20', 24), ('2017-10-21', 24), ('2017-10-22', 24), ('2017-10-23', 24), ('2017-10-24', 24), ('2017-10-25', 24), ('2017-10-26', 24), ('2017-10-27', 24), ('2017-10-28', 24), ('2017-10-29', 24), ('2017-10-30', 24), ('2017-10-31', 24), ('2017-11-01', 24), ('2017-11-02', 24), ('2017-11-03', 24), ('2017-11-04', 24), ('2017-11-05', 24), ('2017-11-06', 24), ('2017-11-07', 24), ('2017-11-08', 24), ('2017-11-09', 24), ('2017-11-10', 24), ('2017-11-11', 24), ('2017-11-12', 24), ('2017-11-13', 24), ('2017-11-14', 24),

데이터를 추가한 다음에 btc_24h_per.csv, btc_24h_sharp.csv, btc_24h.csv 파일을 각각 업데이트를 시작한다.

In [11]:
#btc_24h.csv파일 업데이트
df = pd.read_csv("dataset/krw_btc.csv")
dff = df['open']
dff_array = np.array(dff)
dff_array = dff_array.reshape(2358,24)
col_name =["price0","price1","price2","price3","price4","price5","price6","price7","price8","price9","price10","price11","price12","price13","price14","price15","price16","price17","price18","price19","price20","price21","price22","price23"]
dff_df = pd.DataFrame(dff_array,columns=col_name)
df_24h = dff_df

#btc_24h_per.csv파일 업데이트
dfdf = dff_df.shift(-1)
price_per_df = 100*(dfdf-dff_df)/dff_df
price_per_df.drop([len(price_per_df)-1],axis=0,inplace=True)
price_per_df

#btc_24h_sharp.csv파일 업데이트
warnings.filterwarnings(action='ignore')
col_name =["price0","price1","price2","price3","price4","price5",\
           "price6","price7","price8","price9","price10","price11",\
            "price12","price13","price14","price15","price16","price17",\
            "price18","price19","price20","price21","price22","price23"]

start_time = 1
end_time = len(df_24h)

start_money = df_24h['price0'][start_time]
simul_list = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

btc = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
money = []

for i in range(len(simul_list)):
    simul_list[i].append(start_money)
    money.append(start_money)

for j in range(start_time+1,end_time):
    for i in range(len(simul_list)):
        if df_24h[col_name[i]][j] >= df_24h[col_name[i]][j-1] or (df_24h[col_name[i]][j]-df_24h[col_name[i]][j-1])/df_24h[col_name[i]][j-1] < -0.1:
            if money[i] != 0:
                simul_list[i].append(int(money[i]))
                btc[i] = money[i]/float(df_24h[col_name[i]][j])*0.9995
                money[i] = 0
            
            else:
               simul_list[i].append(int(btc[i]*df_24h[col_name[i]][j]))
               
        elif df_24h[col_name[i]][j] < df_24h[col_name[i]][j-1]:
            if btc[i] != 0:
                money[i] = btc[i]*float(df_24h[col_name[i]][j])*0.9995
                simul_list[i].append(int(money[i]))
                btc[i] = 0
            else:
                simul_list[i].append(int(money[i]))

simul_array = np.array(simul_list)
simul_df = pd.DataFrame(simul_array.T, columns=col_name)
simul_df['open'] = list(df_24h['price0'][start_time:end_time])

simul_log_df =np.log10(simul_df/simul_df['open'][0])

dfdf = simul_df.shift(1)
simul_per_df = 100*(simul_df-dfdf)/dfdf

for i in range(len(simul_list)):
    simul_per_df[col_name[i]][0]= 0.000000
simul_per_df['open'][0]=0.000000


final_list = []
a = end_time - start_time
for j in range(2,a):
    final = np.zeros(25)
    simul_mini_per_df = simul_per_df[:j+1].copy()
    for i in range(len(simul_list)):
        if simul_mini_per_df[col_name[i]][len(simul_mini_per_df)-1] == 0.000000:
            simul_mini_per_df[col_name[i]][len(simul_mini_per_df)-1] = -0.000001
    for k in range(1,j):
        final += np.array(simul_mini_per_df[k:].mean()/simul_mini_per_df[k:].std())
    final_now = final/(j-1)
    final_list.append(final_now)
    
    print(j,"th finish!")
final_array = np.array(final_list)
final_df = pd.DataFrame(final_array, columns = col_name + ['open'])
final_df

2 th finish!
3 th finish!
4 th finish!
5 th finish!
6 th finish!
7 th finish!
8 th finish!
9 th finish!
10 th finish!
11 th finish!
12 th finish!
13 th finish!
14 th finish!
15 th finish!
16 th finish!
17 th finish!
18 th finish!
19 th finish!
20 th finish!
21 th finish!
22 th finish!
23 th finish!
24 th finish!
25 th finish!
26 th finish!
27 th finish!
28 th finish!
29 th finish!
30 th finish!
31 th finish!
32 th finish!
33 th finish!
34 th finish!
35 th finish!
36 th finish!
37 th finish!
38 th finish!
39 th finish!
40 th finish!
41 th finish!
42 th finish!
43 th finish!
44 th finish!
45 th finish!
46 th finish!
47 th finish!
48 th finish!
49 th finish!
50 th finish!
51 th finish!
52 th finish!
53 th finish!
54 th finish!
55 th finish!
56 th finish!
57 th finish!
58 th finish!
59 th finish!
60 th finish!
61 th finish!
62 th finish!
63 th finish!
64 th finish!
65 th finish!
66 th finish!
67 th finish!
68 th finish!
69 th finish!
70 th finish!
71 th finish!
72 th finish!
73 th finish!


,price0,price1,price2,price3,price4,price5,price6,price7,price8,price9,...,price15,price16,price17,price18,price19,price20,price21,price22,price23,open
0,0.707107,0.707107,0.707107,0.707107,0.707107,-0.707107,0.707107,0.707107,0.707107,-0.707107,...,-0.707107,-0.707107,-0.707107,-0.707107,-0.707107,0.707107,0.707107,0.707107,0.707107,1.089200
1,1.044429,1.638535,4.985214,1.684360,1.090216,-0.642230,1.149598,0.977657,1.124956,-0.642229,...,-0.642229,-0.642229,-0.642229,-0.642229,-0.642229,2.612221,3.318763,1.436111,1.257482,1.463067
2,11.142372,2.341310,4.460120,1.598944,1.340190,0.498128,1.767736,1.208462,0.763781,0.440366,...,0.155245,0.114449,0.192193,0.212720,0.211067,2.047774,1.401505,1.507050,10.049636,11.467603
3,3.971629,2.548800,7.651531,2.422659,1.894736,0.981184,1.649755,1.564042,1.218278,0.746402,...,1.140721,0.968240,1.377436,1.135701,1.270144,4.489055,1.780061,20.938362,2.324497,4.212044
4,2.598126,3.868764,4.725678,3.473894,2.623398,1.522039,2.209740,17.182694,1.140681,1.860228,...,0.381089,0.384293,0.151981,0.275059,0.162475,0.399523,0.434715,0.287574,0.256367,2.790607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,0.102597,0.091633,0.081944,0.099561,0.093334,0.096713,0.098019,0.118094,0.108757,0.108422,...,0.082775,0.082886,0.086586,0.093469,0.093442,0.094577,0.089839,0.082096,0.095159,0.095088
2351,0.093510,0.089384,0.084514,0.100912,0.092541,0.095998,0.097321,0.117286,0.108061,0.107754,...,0.082113,0.082282,0.086652,0.092397,0.092724,0.093787,0.089738,0.081363,0.094426,0.086896
2352,0.092784,0.088549,0.087317,0.106480,0.093847,0.097300,0.098212,0.118228,0.109712,0.108675,...,0.081537,0.084675,0.088303,0.091652,0.094039,0.094286,0.091106,0.082708,0.095576,0.088397
2353,0.094820,0.089689,0.088238,0.104076,0.096802,0.100450,0.101348,0.121103,0.113315,0.111657,...,0.084526,0.088514,0.092551,0.094457,0.097029,0.096598,0.101028,0.087204,0.103154,0.094013
